In [1]:
%load_ext autoreload

%autoreload 2

In [30]:
import textwrap

def print_(message: str):

    print(textwrap.fill(message, 75))

In [24]:
import main_agent as ma
import prompts_lib as pl


In [25]:
from langfuse import Langfuse
from langfuse.langchain import CallbackHandler

langfuse = Langfuse(
    public_key="pk-lf-61f55889-2dd2-47a1-ab49-99c60b7ddb40",
    secret_key="sk-lf-632ab357-d225-421f-9f75-7f6685960c50",
    host="https://cloud.langfuse.com"
)

langfuse_handler = CallbackHandler()

In [26]:
logfuse = False

class BasicAgent:
    def __init__(self):
        self.agent = ma.agent
        self.responses = []
    def __call__(self, question: str) -> str:
        print(f"Agent received question (first 50 chars): {question[:50]}...")

        # message = pl.system_prompt2 + question
        if logfuse:
            response = self.agent.invoke(
                input={'messages': question},
                config={"callbacks": [langfuse_handler]}
                )
        else:
            response = self.agent.invoke(input={'messages': question})
        
        self.responses.append(response)


        answer = response.get('messages')[-1].content

        # fixed_answer = "This is a default answer."
        print(f"Agent returning answer: {answer}")
        return answer

In [27]:
agent = BasicAgent()

In [28]:
question = """
How many studio albums were published by Mercedes Sousa between 2000 and 2009 included?
 use duckduckgo tool you have available to search wikipedia to respond
"""

question = """
How many studio albums were published by Mercedes Sousa between 2000 and 2009 included?
you can use the latest 2022 version of the english wikipedia
"""

answer = agent(question)

Agent received question (first 50 chars): 
How many studio albums were published by Mercedes...
Agent returning answer: Three


ToolMessage(content='In the early 1970s, Sosa released two concept albums in collaboration with composer Ariel Ramírez and lyricist Félix Luna: Cantata Sudamericana and Mujeres Argentinas (Argentine Women). She also recorded a tribute to Chilean musician Violeta Parra in 1971, including what was to become one of Sosa\'s signature songs, Gracias a la vida. She further popularized of songs written by Milton Nascimento of Brazil and Pablo Milanés and Silvio Rodríguez both from Cuba. Throughout the decade, she released albums such as Hasta la Victoria in 1972 and Traigo un Pueblo en mi Voz in 1973. They featured songs like "Cuando tenga la tierra", written by Ariel Petrocelli and Daniel Toro, which tackles political and social issues like wealth and land inequality. During the 1970s she was a part of two films by the director Leopoldo Torre Nilsson: El Santo de la Espada in 1970 and Güemes, la tierra en armas in 1971, in which she portrayed Juana Azurduy de Padilla, the guerrilla military 

In [31]:
for m in agent.responses[-1].get('messages'):
    print_(f'{type(m)}: {m.content}')

<class 'langchain_core.messages.human.HumanMessage'>:  How many studio
albums were published by Mercedes Sousa between 2000 and 2009 included? you
can use the latest 2022 version of the english wikipedia
<class 'langchain_core.messages.ai.AIMessage'>:
<class 'langchain_core.messages.tool.ToolMessage'>: In the early 1970s,
Sosa released two concept albums in collaboration with composer Ariel
Ramírez and lyricist Félix Luna: Cantata Sudamericana and Mujeres
Argentinas (Argentine Women). She also recorded a tribute to Chilean
musician Violeta Parra in 1971, including what was to become one of Sosa's
signature songs, Gracias a la vida. She further popularized of songs
written by Milton Nascimento of Brazil and Pablo Milanés and Silvio
Rodríguez both from Cuba. Throughout the decade, she released albums such
as Hasta la Victoria in 1972 and Traigo un Pueblo en mi Voz in 1973. They
featured songs like "Cuando tenga la tierra", written by Ariel Petrocelli
and Daniel Toro, which tackles politi

In [ ]:
message = pl.system_prompt2 + 'How many studio albums were published by Mercedes Sousa between 2000 and 2009 included? use the duckduckgo toll you have available to answer the question'
# message = 'How many studio albums were published by Mercedes Sousa between 2000 and 2009 included? use the duckduckgo toll you have available to answer the question'

message = """
use the duckduckgo tool you have available to search wikipedia to respond:
 How many studio albums were published by Mercedes Sousa between 2000 and 2009 included?"

"""


for step in ma.agent.stream(
    {"messages": [{"role": "user", "content": message}]}, 
    config={"callbacks": [langfuse_handler]}
    ):
    print("="*40)
    print("Current Node:", step)

Current Node: {'assistant': {'messages': [AIMessage(content='I currently don\'t have access to live tools like DuckDuckGo or the internet to perform searches. However, I can provide information based on my existing knowledge. Could you clarify if you meant "Mercedes Sosa," the famous Argentine singer? If so, I can help you with her discography details based on my knowledge. Let me know!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 42, 'total_tokens': 111, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-CFtQQV0iFj7IB32vT0dZP4GQXBCTa', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'

In [ ]:
user_input="Hi there!, My name is subitha, I am a software engineer. Can you tell me about the latest research on quantum computing?"

events=graph.stream(
     {"messages": [("user", user_input)]},stream_mode="values"
)

for event in events:
  event["messages"][-1].pretty_print()